<a href="https://colab.research.google.com/github/anggaangoro3/Prediksi-Harga-Tanah-Jakarta/blob/main/Scrping_Rumah123_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Chrome and Chromedriver
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/chromedriver

print("Chrome and Chromedriver installed and configured.")

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Chrome and Chromedriver installed and configured.


'cp' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import random
import pandas as pd
import os
import threading
import tkinter as tk
from tkinter import messagebox

# --- Pesan Perkenalan ---
print("Research by @Reza Anggoro")
print("Use it for Educational Purposes only!")
print("\nThis script uses Chromium Browser to crawl data from Rumah123.")
print("Note: This script is configured to run on your local device.")
print("\nOpening rumah123 search page...\n")
print("Note: A Stop button window will appear. Click it to stop scraping and save data.\n")

# --- Pengaturan Opsi Chrome untuk LOCAL (NON-HEADLESS) ---
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")  # Membuka browser fullscreen

# --- Menambahkan User Agent Acak ---
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36'
]
options.add_argument(f'user-agent={random.choice(user_agents)}')

# --- Inisialisasi Driver Chrome ---
print("Starting local Chrome driver...")
driver = webdriver.Chrome(options=options)
print("Driver started successfully.")

# List untuk menyimpan data hasil scrape
data_list = []

# Path file CSV
csv_file = './scraped_land_data.csv'

# --- Pengecekan File CSV yang Sudah Ada ---
if os.path.exists(csv_file):
    old_file = csv_file.replace('.csv', '.old.csv')
    try:
        os.rename(csv_file, old_file)
        print(f"Found existing file {csv_file}, renaming to {old_file}")
    except OSError as e:
        print(f"Could not rename file {csv_file}. It might be open. Error: {e}")
        print("Exiting to prevent data loss.")
        driver.quit()
        exit()

# URL dasar
base_url = "https://www.rumah123.com/jual/jakarta-pusat/tanah/?page="

# Rentang halaman yang akan di-scrape
start_page = 2
end_page = 99

# Flag untuk stop scraping
stop_scraping = False

# Fungsi untuk window tombol stop (jalan di thread terpisah)
def stop_button_window():
    global stop_scraping
    root = tk.Tk()
    root.title("Scraping Control")
    root.geometry("200x100")

    def on_stop():
        global stop_scraping
        stop_scraping = True
        messagebox.showinfo("Stop", "Scraping will stop after current operation.")
        root.destroy()

    button = tk.Button(root, text="Stop Scraping", command=on_stop)
    button.pack(pady=20)
    root.mainloop()

# Jalankan window stop di thread background
threading.Thread(target=stop_button_window, daemon=True).start()

# --- Loop Halaman Utama (PAGINATION) ---
for page in range(start_page, end_page + 1):
    if stop_scraping:
        break

    url = base_url + str(page)
    print(f"Opening page {page}: {url}")
    driver.get(url)

    # Jeda acak untuk memuat halaman
    time.sleep(random.uniform(3, 4.5))

    # --- Pengecekan CAPTCHA yang Terlihat ---
    captcha_detected = False
    try:
        captcha_element = driver.find_element(By.CSS_SELECTOR, 'iframe[src*="recaptcha"], div.g-recaptcha')
        if captcha_element.is_displayed():
            captcha_detected = True
    except NoSuchElementException:
        pass

    if captcha_detected:
        print(f"!!! CAPTCHA DETECTED on page {page}. !!!")
        print("!!! ANDA HARUS MENYELESAIKANNYA DI JENDELA BROWSER SEKARANG. !!!")
        print("!!! Skrip akan menjeda selama 20 DETIK agar Anda bisa menyelesaikannya. !!!")
        time.sleep(20)
        print("Waktu jeda selesai, melanjutkan skrip...")

    # --- Blok untuk Scrolling ---
    print("Waiting for page layout to settle before scrolling...")
    time.sleep(1.5)

    total_height = driver.execute_script("return document.body.scrollHeight")
    current_scroll = 0
    scroll_count = 1
    max_scroll_attempts = 50
    print("Starting scroll (slower and deeper)...")

    while current_scroll < total_height and scroll_count < max_scroll_attempts:
        if stop_scraping:
            break

        scroll_amount = random.randint(400, 700)
        driver.execute_script(f"window.scrollBy(0, {scroll_amount});")

        print(f"-- Scrolling... ({scroll_count}), waiting for content...")
        time.sleep(random.uniform(1.5, 2.2))

        current_scroll += scroll_amount
        scroll_count += 1

        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == total_height:
            print("-- Page height not changing, likely at bottom.")
            break
        total_height = new_height

    if stop_scraping:
        break

    print("Doing one final scroll to the absolute bottom...")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    print("Waiting 3 seconds at the bottom for final loads...")
    time.sleep(3.0)

    print("Experiment: Scrolling back to top slowly...")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(random.uniform(1.5, 2.5))

    mid_height = total_height // 2
    print(f"Experiment: Scrolling to middle ({mid_height}px) and pausing...")
    driver.execute_script(f"window.scrollTo(0, {mid_height});")
    time.sleep(random.uniform(2.0, 3.0))

    print("Scrolling finished for this page.")

    # --- Ambil semua link (href) dari halaman daftar ---
    href_list = []
    try:
        listings = driver.find_elements(By.CSS_SELECTOR, "a.gap-1.w-full")
        for listing in listings:
            try:
                href = listing.get_attribute('href')
                if href and href.startswith('https://www.rumah123.com/properti/'):
                    href_list.append(href)
            except:
                pass
        href_list = list(dict.fromkeys(href_list))
        print(f"Found {len(href_list)} property links on page {page}.")
    except NoSuchElementException:
        print(f"No more listings found on page {page}. Stopping.")
        break

    if not href_list:
        print(f"No valid listings found on page {page}. Stopping.")
        break

    # --- Loop Halaman Detail (per link) ---
    for href in href_list:
        if stop_scraping:
            break

        time.sleep(random.uniform(1, 3))
        print(f"Opening detail page: {href}")
        driver.get(href)
        time.sleep(random.uniform(3, 4.5))

        # Cek VISIBLE CAPTCHA di halaman detail
        captcha_detected = False
        try:
            captcha_element_detail = driver.find_element(By.CSS_SELECTOR, 'iframe[src*="recaptcha"], div.g-recaptcha')
            if captcha_element_detail.is_displayed():
                captcha_detected = True
        except NoSuchElementException:
            pass

        if captcha_detected:
            print(f"!!! CAPTCHA DETECTED on detail page {href}. !!!")
            print("!!! Skrip akan menjeda selama 20 DETIK agar Anda bisa menyelesaikannya. !!!")
            time.sleep(20)
            print("Waktu jeda selesai, melanjutkan skrip...")

        # Coba klik tombol "Muat lebih banyak" (jika ada)
        try:
            expand_container = driver.find_element(By.CSS_SELECTOR, "#property-information > div:nth-of-type(1) > div:nth-of-type(2)")
            expand_button = expand_container.find_element(By.CSS_SELECTOR, "span[data-test-id='expanded-specification']")
            ActionChains(driver).move_to_element(expand_button).perform()
            time.sleep(random.uniform(0.5, 1))
            expand_button.click()
            time.sleep(random.uniform(1, 2))
            print("Successfully clicked 'Muat lebih banyak' button.")
        except Exception as e:
            print(f"No 'Muat lebih banyak' button found or click failed. Proceeding without expansion.")

        # --- Blok Ekstraksi Data (DIPERBAIKI: Tambah try-except lengkap) ---
        data = {}
        try:
            data['Product_sku_0'] = driver.find_element(By.CSS_SELECTOR, "div.flex:nth-of-type(4) p.text-sm").text
        except:
            data['Product_sku_0'] = ''

        try:
            data['Place_name_0'] = driver.find_element(By.TAG_NAME, "h1").text
        except:
            data['Place_name_0'] = ''

        try:
            data['Total_Price_0'] = driver.find_element(By.CSS_SELECTOR, "span.text-primary").text
        except:
            data['Total_Price_0'] = ''

        try:
            data['Price_per_meter_0'] = driver.find_element(By.CSS_SELECTOR, "span.font-normal.text-sm").text
        except:
            data['Price_per_meter_0'] = ''

        try:
            data['Land_Area_0'] = driver.find_element(By.CSS_SELECTOR, "div.flex-col:nth-of-type(1) p.text-gray-800").text
        except:
            data['Land_Area_0'] = ''

        try:
            data['Certificate_0'] = driver.find_element(By.CSS_SELECTOR, "div.flex:nth-of-type(2) p.text-base").text
        except:
            data['Certificate_0'] = ''

        try:
            data['Land_Dimensions_0'] = driver.find_element(By.CSS_SELECTOR, "div.flex:nth-of-type(1) p.font-medium.text-sm").text
        except:
            data['Land_Dimensions_0'] = ''

        try:
            data['Property_Type_0'] = driver.find_element(By.CSS_SELECTOR, "div.flex:nth-of-type(2) p.font-medium.text-sm").text
        except:
            data['Property_Type_0'] = ''

        try:
            data['Ad_Type_0'] = driver.find_element(By.CSS_SELECTOR, "div.items-center:nth-of-type(3) p.text-sm").text
        except:
            data['Ad_Type_0'] = ''

        try:
            data['Place_PostalAddress_addressLocality_0'] = driver.find_element(By.CSS_SELECTOR, "p.mb-2").text
        except:
            data['Place_PostalAddress_addressLocality_0'] = ''

        try:
            data['BreadcrumbList_ListItem_name_2'] = driver.find_element(By.CSS_SELECTOR, ".flex div:nth-of-type(4) a").text
        except:
            data['BreadcrumbList_ListItem_name_2'] = ''

        try:
            data['BreadcrumbList_ListItem_name_3'] = driver.find_element(By.CSS_SELECTOR, ".flex div:nth-of-type(5) a").text
        except:
            data['BreadcrumbList_ListItem_name_3'] = ''

        try:
            data['Product_description_0'] = driver.find_element(By.CSS_SELECTOR, "p.font-light.text-sm").text
        except:
            data['Product_description_0'] = ''

        # Print data untuk debug
        print(f"Extracted data: {data}")

        data_list.append(data)

        # Simpan data ke CSV secara bertahap
        df = pd.DataFrame(data_list)
        df.to_csv(csv_file, index=False)
        print(f"Your data saved to: {csv_file}")
        print(f"Total items saved: {len(data_list)}\n")

        # Jeda sebelum kembali ke halaman daftar
        time.sleep(random.uniform(1, 3))
        print(f"Going back to listing page {page}...")
        driver.get(url)
        time.sleep(random.uniform(1, 2))

    # Jeda antar halaman
    print(f"--Finished page {page}. Taking a break, waiting for 10 seconds...\n")
    time.sleep(5)

# --- Selesai ---
driver.quit()

# Simpan data terakhir jika belum
if data_list:
    df = pd.DataFrame(data_list)
    df.to_csv(csv_file, index=False)
    print(f"Final data saved to: {csv_file}")
    print(f"Total items saved: {len(data_list)}")

print("Scraping completed (or stopped). Data saved to 'scraped_land_data.csv'.")

Research by @Reza Anggoro
Use it for Educational Purposes only!

This script uses Chromium Browser to crawl data from Rumah123.
Note: This script is configured to run on your local device.

Opening rumah123 search page...

Note: A Stop button window will appear. Click it to stop scraping and save data.

Starting local Chrome driver...
Driver started successfully.
Opening page 2: https://www.rumah123.com/jual/jakarta-pusat/tanah/?page=2
Waiting for page layout to settle before scrolling...
Starting scroll (slower and deeper)...
-- Scrolling... (1), waiting for content...
-- Page height not changing, likely at bottom.
Doing one final scroll to the absolute bottom...
Waiting 3 seconds at the bottom for final loads...
Experiment: Scrolling back to top slowly...
Experiment: Scrolling to middle (9011px) and pausing...
Scrolling finished for this page.
Found 20 property links on page 2.
Opening detail page: https://www.rumah123.com/properti/jakarta-pusat-hasyim-ashari/for-sale-kavling-hasyim-

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.60); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff601557a05
	0x7ff601557a60
	0x7ff6012d16ad
	0x7ff6012bd1c5
	0x7ff6012e2a5a
	0x7ff60135a306
	0x7ff60137b222
	0x7ff60131b068
	0x7ff60131be93
	0x7ff6018129a0
	0x7ff60180ce20
	0x7ff60182cc15
	0x7ff60157309e
	0x7ff60157ad8f
	0x7ff601560be4
	0x7ff601560d9f
	0x7ff6015467f8
	0x7fffc81f7374
	0x7fffc8d1cc91


In [ ]:
import pandas as pd

def read_scraped_data(filename='scraped_land_data.csv'):
    """Reads scraped data from a CSV file into a pandas DataFrame."""
    try:
        df = pd.read_csv(filename)
        print(f"Successfully loaded data from '{filename}'.")
        print(f"DataFrame shape: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Warning: The file '{filename}' is empty or only contains headers.")
        return pd.DataFrame() # Return an empty DataFrame
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

scraped_df = read_scraped_data()

if scraped_df is not None and not scraped_df.empty:
    print("Displaying the entire scraped dataset:")
    display(scraped_df)
elif scraped_df is not None and scraped_df.empty:
    print("The dataset is empty. No data to display.")
else:
    print("Could not read the dataset.")

Successfully loaded data from 'scraped_land_data.csv'.
DataFrame shape: (340, 13)
Displaying the entire scraped dataset:


,Product_sku_0,Place_name_0,Total_Price_0,Price_per_meter_0,Land_Area_0,Certificate_0,Land_Dimensions_0,Property_Type_0,Ad_Type_0,Place_PostalAddress_addressLocality_0,BreadcrumbList_ListItem_name_2,BreadcrumbList_ListItem_name_3,Product_description_0
0,NaN,Dijual Tanah Prime Lokasi Pondok Indah Jakarta...,"Rp 18,8 Miliar Total",Rp 42 Juta /m²,448 m²,SHM,Tanah,Dijual,las8927165,"Pondok Indah, Jakarta Selatan",Jakarta Selatan,Pondok Indah,Dijual Tanah Prime lokasi di Pondok Indah Jaka...
1,NaN,Tanah Kavling di Pondok Indah Jakarta Selatan ...,"Rp 18,8 Miliar Total",Rp 42 Juta /m²,448 m²,PPJB,Tanah,Dijual,las8934406,"Pondok Indah, Jakarta Selatan",Jakarta Selatan,Pondok Indah,Tanah Kavling di Pondok Indah Jakarta Selatan ...
2,NaN,Tanah Kavling Ebony Termurah di Green Lake Cit...,"Rp 3,2 Miliar Total","Rp 17,8 Juta /m²",180 m²,PPJB,Tanah,Dijual,las8894248,"Green Lake City, Jakarta Barat",Jakarta Barat,Green Lake City,Tanah Kavling Ebony Termurah di Green Lake Cit...
3,NaN,Dijual Kavling Citra Garden Puri Luas 156 M2 Hoek,"Rp 2,02 Miliar Total",Rp 13 Juta /m²,156 m²,Lainnya,Tanah,Dijual,las4203235,"Citra Garden, Jakarta Barat",Jakarta Barat,Citra Garden,DIJUAL\n\nKavling Citra Garden Puri\nCluster C...
4,NaN,Dijual Kavling Pinisi Indah Pik Luas 620 M2 Bo...,Rp 21 Miliar Total,Rp 34 Juta /m²,620 m²,SHM,Tanah,Dijual,las4204357,"Pantai Indah Kapuk, Jakarta Utara",Jakarta Utara,Pantai Indah Kapuk,DIJUAL\n\nKavling Pinisi Indah PIK\nBoulevard\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,NaN,Jual Tanah Plus Bangunan Kedoya Selatan Jakart...,Rp 10 Miliar Total,NaN,207 m² (18x12m),SHM,18x12 m,Tanah,Dijual,"Kedoya Selatan, Jakarta Barat",Jakarta Barat,Kedoya Selatan,DIJUAL CEPAT !!\n\nTANAH PLUS BANGUNAN\nKedoya...
336,NaN,Kavling Siap Bangum Best And Strategic Area La...,Rp 910 Juta Total,Rp 13 Juta /m²,70 m²,SHM,Tanah,Dijual,las8870044,"Kebagusan, Jakarta Selatan",Jakarta Selatan,Kebagusan,Kavling siap bangum Best and Strategic Area la...
337,NaN,Dijual Tanah Kavling Lokasi Strategis Di Sreng...,"Rp 2,3 Miliar Total",NaN,150 m² (7x21m),SHM,7x21 m,Tanah,Dijual,"Srengseng, Jakarta Barat",Jakarta Barat,Srengseng,Tanah di Srengseng.\n\nTemukan tanah 1 lantai ...
338,NaN,Kavling Murah Di Keutamaan Gajah Mada Jakarta ...,"Rp 5,7 Miliar",NaN,214 m² (26x8m),HGB,26x8 m,Tanah,Dijual,"Tamansari, Jakarta Barat",Jakarta Barat,Tamansari,Tanah ngantong lebar belakang. \nluas tanah 21...
